# Notebook

In [119]:
import math
import yaml

## Book Keeping

In [120]:
MODEL = "PE1R_112_MX4"
VERBOSITY = 1

In [121]:
SWITCHING_FREQUENCY = 2000
BUS_VOLTAGE = 650
PEAK_CARRIER_VOLTAGE = 3.25
GEARBOX_GAIN = 10
CURRENT_SENSOR_GAIN = 1 / 90
CURRENT_RESPONSE_TIME = 50e-3
CURRENT_OVERSHOOT = 7
SPEED_SENSOR_GAIN =  2 / 100
SPEED_RESPONSE_TIME = 500e-3
SPEED_OVERSHOOT = 5
POSITION_SENSOR_GAIN =  1 / 120
POSITION_RESPONSE_TIME = 5000e-3
POSITION_OVERSHOOT = 12

In [122]:
with open("../reports/" + MODEL + "_parameters.yaml", "r") as file:
    parameters = yaml.safe_load(file)

In [123]:
Tn = parameters["Tn"]
wn = parameters["wn"]
Br = parameters["Br"]
Jr = parameters["Jr"]

T = Tn * 0.9
Ko = (T * 0.8) * GEARBOX_GAIN

Beq = (T * 0.2) / wn
Bl = (Beq - Br) * GEARBOX_GAIN**2

Jeq = Beq * 3
Jl = (Jeq - Br) * GEARBOX_GAIN**2

In [124]:
Kc = 2 * BUS_VOLTAGE / PEAK_CARRIER_VOLTAGE
Ts = 1 / SWITCHING_FREQUENCY

if VERBOSITY:
    print("Kc :: {0}".format(Kc))

Kc :: 400.0


In [125]:
Rs = parameters["Rs"]
Ld = parameters["Lq"]

tau_maid = Ld / Rs
Kmaid = (Kc * CURRENT_SENSOR_GAIN) / Rs
tau_mfid = CURRENT_RESPONSE_TIME / (3 * 10)
zeta_id = -math.log(CURRENT_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(CURRENT_OVERSHOOT / 100)**2)
Kpid = ((2 * tau_maid) / tau_mfid - 1) / Kmaid
Tiid = (tau_mfid**2 * zeta_id**2 * Kmaid) / tau_maid
tau_zid = Kpid * Tiid

if VERBOSITY:
    print("Kpid :: {:.4e}".format(Kpid))
    print("Tiid :: {:.4e}".format(Tiid))
    print("tau_zid :: {:.4e}".format(tau_zid))

Kpid :: 3.6608e+00
Tiid :: 3.6549e-04
tau_zid :: 1.3380e-03


In [126]:
Lq = parameters["Lq"]

tau_maiq = Lq / Rs
Kmaiq = (Kc * CURRENT_SENSOR_GAIN) / Rs
tau_mfiq = CURRENT_RESPONSE_TIME / 3
zeta_iq = -math.log(CURRENT_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(CURRENT_OVERSHOOT / 100)**2)
Kpiq = ((2 * tau_maiq) / tau_mfiq - 1) / Kmaiq
Tiiq = (tau_mfiq**2 * zeta_iq**2 * Kmaiq) / tau_maiq
tau_ziq = Kpiq * Tiiq

if VERBOSITY:
    print("Kpiq :: {:.4e}".format(Kpiq))
    print("Tiiq :: {:.4e}".format(Tiiq))
    print("tau_ziq :: {:.4e}".format(tau_ziq))

Kpiq :: 2.3445e-01
Tiiq :: 3.6549e-02
tau_ziq :: 8.5688e-03


In [127]:
Ke = parameters["Ke"]
Kt = parameters["Kt"]

Jti = (Ke * Kt * Tiiq) / (Kc * CURRENT_SENSOR_GAIN)
Ksv = SPEED_SENSOR_GAIN * (30 / math.pi) 

tau_mav = (Jeq + Jti) / Beq
Kmav = (Kt * Ksv) / (Beq * GEARBOX_GAIN * CURRENT_SENSOR_GAIN)
tau_mfv = SPEED_RESPONSE_TIME / 3
zeta_v = -math.log(SPEED_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(SPEED_OVERSHOOT / 100)**2)
Kpv = ((2 * tau_mav) / tau_mfv - 1) / Kmav
Tiv = (tau_mfv**2 * zeta_v**2 * Kmav) / tau_mav
tau_zv = Kpv * Tiv

if VERBOSITY:
    print("Kpv :: {:.4e}".format(Kpv))
    print("Tiv :: {:.4e}".format(Tiv))
    print("tau_zv :: {:.4e}".format(tau_zv))


Kpv :: 4.2898e-01
Tiv :: 3.6197e-01
tau_zv :: 1.5527e-01


In [128]:
Ksp = POSITION_SENSOR_GAIN * (180 / math.pi)
Kmap = Ksp / Ksv
tau_mfp = POSITION_RESPONSE_TIME / 3
Kpp = 1 / (Kmap * tau_mfp)

if VERBOSITY:
    print("Kpp :: {:.4e}".format(Kpp))

Kpp :: 2.4000e-01


In [129]:
zeta_p = -math.log(POSITION_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(POSITION_OVERSHOOT / 100)**2)
Kpp_ = 2 / (tau_mfp * Kmap)
Tip = tau_mfp**2 * zeta_p**2 * Kmap
tau_zp = Kpp_ * Tip

if VERBOSITY:
    print("Kpp' :: {:.4e}".format(Kpp_))
    print("Tip :: {:.4e}".format(Kpp_))
    print("tau_zp :: {:.4e}".format(tau_zp))

Kpp' :: 4.8000e-01
Tip :: 4.8000e-01
tau_zp :: 1.0432e+00


In [130]:
with open("../reports/" + MODEL + "_parameters.yaml", "a") as file:
    documents = yaml.dump({
            "Vcc": BUS_VOLTAGE,
            "Vtpp": PEAK_CARRIER_VOLTAGE,
            "fs": SWITCHING_FREQUENCY,
            "Kg": GEARBOX_GAIN,
            "Bl": Bl,
            "Jl": Jl,
            "Ksi": CURRENT_SENSOR_GAIN,
            "Ksv": SPEED_SENSOR_GAIN,
            "Ksp": POSITION_SENSOR_GAIN,
            "Kc": Kc,
            "Kpi_d": Kpid,
            "Tii_d": Tiid,
            "tau_zi_d": tau_zid,
            "Kpi_q": Kpiq,
            "Tii_q": Tiiq,
            "tau_zi_q": tau_ziq,
            "Kpv": Kpv,
            "Tiv": Tiv,
            "tau_zv": tau_zv,
            "Kpp": Kpp,
            "Kpp_": Kpp_,
            "Tip": Tip,
            "tau_zp": tau_zp,
            "Ko": Ko,
        }, 
        file
    )